# HMCTuning

Author: <b>Ignacio Peis</b>

*Note*: All the gifs can be obtained using the scripts in '../examples/'


This notebook includes some illustrative examples that will help in understanding why tuning the hyperparameters with HMCTuning is effective. Although some background in Hamiltonian Monte Carlo might be helpful, all you need to know is explained in the Introduction section.


## Introduction


When sampling with HMC, you have to set some hyperparameters:

* Step sizes $\mathbf{\epsilon}$. Matrix with dims $(T,D)$. Different step sizes can be learned to be applied within each state of the chains.
* Momentum variances, $M$. Matrix with dims $(T, D)$.
* An inflation/scale parameter $\mathbf{s}$, that can be a scalar or a vector with dims $D$ so that different inflations can be applied per dimension.


You can observe in these gifs how the initial proposal affects the sampling procedure. The proposal is cyclically changed to show visually demonstrate that, when the proposal is too tight, chains will get stuck in local density regions or modes and will not explore the whole density.
<p float="center">
    <img src="../figs/cycle/gaussian_mixture/samples.gif" width="300" align="center">     &emsp;
    <img src="../figs/cycle/wave/samples.gif" width="300" align="center">
</p>




For the first distribution (Mixture of Gaussians), a proper initial proposal is a zero-centered Gaussian. As you can observe, the chains will reach all the modes:
<p float="center">
    <img src="../figs/chains/gaussian_mixture/samples.gif" width="300" align="center">
</p>

What about the second density ("wave")? By choosing the same centered, low variance proposal, chains will get stuck in a small region:
<p float="center">
    <img src="../figs/chains/wave/samples_stuck.gif" width="300" align="center">
</p>

If we define a wider horizontal variance (for instance, [0.1, 5.0]), the proposal will cover better the wave density. 




So the question is: could we automatically tune that initial proposal? The answer is yes. Let's fit the previous HMC.
